# Descargar el dataset desde kaggle

Para poder descargar el dataset usado en el modelo ejecuta el siguiente comando, ten en cuenta que estamos usando kaggle para esto y debes instalarlo y configurar la apiKey.

`pip install kaggle`

`kaggle datasets download -d grassknoted/asl-alphabet`

`kaggle datasets download datamunge/sign-language-mnist`

#### Mover Imagenes de test a train para hacer la división personalizada, asi cumplir con un 80 train / 20 test

In [1]:
import os
import shutil
import random
import math

In [ ]:
def move_test_images():
    """
    Moves test images to their corresponding letter folder in the train directory.
    """
    source_dir = '../data/raw/asl-alphabet/asl_alphabet_test/asl_alphabet_test'
    train_dir = '../data/raw/asl-alphabet/asl_alphabet_train/asl_alphabet_train'

    if not os.path.exists(source_dir):
        print(f"Error: Source directory not found at {source_dir}")
        return

    try:
        image_files = [f for f in os.listdir(source_dir) if f.endswith('.jpg')]
    except FileNotFoundError:
        print(f"Error: Could not list files in {source_dir}. It might not be a directory.")
        return


    for filename in image_files:
        letter = filename.split('_')[0]

        destination_folder = os.path.join(train_dir, letter)

        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(destination_folder, filename)

        try:
            shutil.move(source_path, destination_path)
            print(f"Moved {filename} to {destination_folder}")
        except FileNotFoundError:
            print(f"Error: Could not find {filename} to move.")
        except Exception as e:
            print(f"An error occurred while moving {filename}: {e}")


move_test_images()
print("Script finished.")


In [4]:
def split_data(source_dir, processed_dir, split_ratio=0.8):
    """
    Splits the data from source_dir into training and testing sets
    and saves them in processed_dir.

    Args:
        source_dir (str): The path to the directory containing the raw data,
                          with subdirectories for each class.
        processed_dir (str): The path to the directory where the processed
                             (split) data will be saved.
        split_ratio (float): The ratio of training data to the total data.
    """
    train_dir = os.path.join(processed_dir, 'train')
    test_dir = os.path.join(processed_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    if not os.path.exists(source_dir):
        print(f"Error: Source directory not found at {source_dir}")
        return

    for letter_folder in os.listdir(source_dir):
        letter_path = os.path.join(source_dir, letter_folder)
        if os.path.isdir(letter_path):
            train_letter_dir = os.path.join(train_dir, letter_folder)
            test_letter_dir = os.path.join(test_dir, letter_folder)
            os.makedirs(train_letter_dir, exist_ok=True)
            os.makedirs(test_letter_dir, exist_ok=True)

            images = [f for f in os.listdir(letter_path) if os.path.isfile(os.path.join(letter_path, f))]
            random.shuffle(images)

            split_point = math.ceil(len(images) * split_ratio)
            train_images = images[:split_point]
            test_images = images[split_point:]

            for image in train_images:
                source_image_path = os.path.join(letter_path, image)
                dest_image_path = os.path.join(train_letter_dir, image)
                shutil.copyfile(source_image_path, dest_image_path)
            
            print(f"Copied {len(train_images)} images to {train_letter_dir}")

            for image in test_images:
                source_image_path = os.path.join(letter_path, image)
                dest_image_path = os.path.join(test_letter_dir, image)
                shutil.copyfile(source_image_path, dest_image_path)

            print(f"Copied {len(test_images)} images to {test_letter_dir}")



SOURCE_DATA_DIR = '../data/raw/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
PROCESSED_DATA_DIR = '../data/processed'
    
split_data(SOURCE_DATA_DIR, PROCESSED_DATA_DIR)
print("Data splitting finished.")


Copied 2401 images to ../data/processed/train/R
Copied 600 images to ../data/processed/test/R
Copied 2401 images to ../data/processed/train/U
Copied 600 images to ../data/processed/test/U
Copied 2401 images to ../data/processed/train/I
Copied 600 images to ../data/processed/test/I
Copied 2401 images to ../data/processed/train/N
Copied 600 images to ../data/processed/test/N
Copied 2401 images to ../data/processed/train/G
Copied 600 images to ../data/processed/test/G
Copied 2401 images to ../data/processed/train/Z
Copied 600 images to ../data/processed/test/Z
Copied 2401 images to ../data/processed/train/T
Copied 600 images to ../data/processed/test/T
Copied 2401 images to ../data/processed/train/S
Copied 600 images to ../data/processed/test/S
Copied 2401 images to ../data/processed/train/A
Copied 600 images to ../data/processed/test/A
Copied 2401 images to ../data/processed/train/F
Copied 600 images to ../data/processed/test/F
Copied 2401 images to ../data/processed/train/O
Copied 600 i

# Normalización y preprocesamiento de datos
1. Normalizar
2. Convertir a escala de grises

In [50]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, InputLayer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


In [13]:
TRAIN_DIR = '../data/processed/train'
TEST_DIR = '../data/processed/test'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

BATCH_SIZE = 32
EPOCHS = 30
IMG_HEIGHT = 200
IMG_WIDTH = 200

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

print("Class indices:", train_generator.class_indices)


Found 69628 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.
Class indices: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}
Found 17400 images belonging to 29 classes.
Class indices: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'del': 26, 'nothing': 27, 'space': 28}


## Arquitectura de la red neuronal

In [4]:
model = keras.Sequential(
    [
        layers.Input(shape=(120000,)),
        layers.Dense(512, activation="relu"),
        layers.Dense(29, activation="softmax"),
    ]
)

model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 512)            │    61,440,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,455,389 (234.43 MB)

 Trainable params: 61,455,389 (234.43 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
num_classes = len(train_generator.class_indices)

model = Sequential([
    InputLayer(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(512, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 97, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 46, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 67712)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │    34,669,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 29)             │        14,877 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,777,181 (132.66 MB)

 Trainable params: 34,777,181 (132.66 MB)

 Non-trainable params: 0 (0.00 B)

## **Entrenamiento y guardado del Modelo**

In [ ]:
!pip install scipy

In [15]:
FILE_PATH = '../models/best_asl_model.keras'
checkpoint = ModelCheckpoint(
    filepath=FILE_PATH,
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[checkpoint]
)
print("Trainning finished, best model saved at:", FILE_PATH)


Epoch 1/30
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 0s 509ms/step - accuracy: 0.2305 - loss: 2.6487

/Users/jepolancos/projects/hands-unmuted/.venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from -inf to 0.78787, saving model to ../models/best_asl_model.keras
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 1172s 538ms/step - accuracy: 0.2306 - loss: 2.6484 - val_accuracy: 0.7879 - val_loss: 0.6132
Epoch 2/30
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 0s 505ms/step - accuracy: 0.7045 - loss: 0.8791
Epoch 2: val_accuracy improved from 0.78787 to 0.90603, saving model to ../models/best_asl_model.keras
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 1160s 533ms/step - accuracy: 0.7045 - loss: 0.8790 - val_accuracy: 0.9060 - val_loss: 0.2632
Epoch 3/30
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 0s 493ms/step - accuracy: 0.8164 - loss: 0.5421
Epoch 3: val_accuracy improved from 0.90603 to 0.94598, saving model to ../models/best_asl_model.keras
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 1136s 522ms/step - accuracy: 0.8164 - loss: 0.5421 - val_accuracy: 0.9460 - val_loss: 0.1558
Epoch 4/30
2176/2176 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step - accuracy: 0.8597 - loss: 0.4074
Epoch 4: val_accuracy did not improve from 0.94598


### **Evaluación del modelo**

In [ ]:
loaded_model = keras.models.load_model(FILE_PATH)
test_loss, test_accuracy = loaded_model.evaluate(validation_generator)
print(f"Test accuracy: {test_accuracy:.4f}")



544/544 ━━━━━━━━━━━━━━━━━━━━ 64s 117ms/step - accuracy: 0.9954 - loss: 0.0132
Test accuracy: 0.9956
544/544 ━━━━━━━━━━━━━━━━━━━━ 64s 117ms/step - accuracy: 0.9954 - loss: 0.0132
Test accuracy: 0.9956


### **Función de predicción del modelo**

In [53]:

try:
    model = keras.models.load_model(FILE_PATH)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()


def predict_image(model, img_path, class_indices):
    """Carga una imagen, la preprocesa y predice su clase."""

    img = image.load_img(img_path, target_size=(200, 200))
    

    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    

    img_array /= 255.0
    

    predictions = model.predict(img_array)
    

    predicted_class_index = np.argmax(predictions[0])
    confidence = np.max(predictions[0])
    


    index_to_class = {v: k for k, v in class_indices.items()}
    
    predicted_class_name = index_to_class[predicted_class_index]
    
    print(f"Índice de clase predicho: {predicted_class_index}, Confianza: {confidence:.4f}")
    print(f"Nombre de clase predicho: {predicted_class_name}")
    
    return predicted_class_name

try:
    image_path = '../data/test_images/2_test.jpg'
    predicted_class = predict_image(model, image_path, train_generator.class_indices)
    print(f"\nResultado final para {image_path}: {predicted_class}")
except NameError:
    print("Asegúrate de haber ejecutado la celda que define 'train_generator' primero.")
except Exception as e:
    print(f"Ocurrió un error durante la predicción: {e}")



Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Índice de clase predicho: 9, Confianza: 0.9999
Nombre de clase predicho: J

Resultado final para ../data/test_images/2_test.jpg: J
Índice de clase predicho: 9, Confianza: 0.9999
Nombre de clase predicho: J

Resultado final para ../data/test_images/2_test.jpg: J
